# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nguigmi,14.2495,13.1092,37.04,30,38,2.64,NE,1720101275
1,1,yola,9.2000,12.4833,35.04,38,80,2.84,NG,1720101277
2,2,tucumcari,35.1717,-103.7250,20.91,64,0,2.57,US,1720101279
3,3,udachny,66.4167,112.4000,13.91,47,5,2.38,RU,1720101281
4,4,manicore,-5.8092,-61.3003,32.50,50,35,1.07,BR,1720101283


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', color='City', alpha=0.6, 
    hover_cols=['City', 'Humidity', 'Country']
)
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) & 
    (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,capelinha,-17.6914,-42.5158,21.89,42,0,2.95,BR,1720101361
87,87,shiraoi,42.5500,141.3500,21.99,94,0,2.35,JP,1720101444
112,112,newman,37.3138,-121.0208,24.60,30,0,1.30,US,1720101494
310,310,holualoa,19.6228,-155.9522,22.15,81,0,2.57,US,1720101878
365,365,eastham,41.8301,-69.9739,25.03,76,0,4.12,US,1720101985


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
45,capelinha,BR,-17.6914,-42.5158,42,
87,shiraoi,JP,42.5500,141.3500,94,
112,newman,US,37.3138,-121.0208,30,
310,holualoa,US,19.6228,-155.9522,81,
365,eastham,US,41.8301,-69.9739,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    #"categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
capelinha - nearest hotel: No hotel found
shiraoi - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
eastham - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
paraisopolis - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
sanming - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
45,capelinha,BR,-17.6914,-42.5158,42,No hotel found
87,shiraoi,JP,42.5500,141.3500,94,No hotel found
112,newman,US,37.3138,-121.0208,30,No hotel found
310,holualoa,US,19.6228,-155.9522,81,No hotel found
365,eastham,US,41.8301,-69.9739,76,No hotel found
500,howard springs,AU,-12.4970,131.0470,54,No hotel found
507,paraisopolis,BR,-22.5542,-45.7800,48,No hotel found
518,tolanaro,MG,-25.0319,46.9987,80,No hotel found
583,sanming,CN,26.2486,117.6186,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', color='City', alpha=0.6, 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)